In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2" 
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/zechen/.conda/envs/kgmaml/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import torch
import torch.nn as nn

import numpy as np
import random

pad = tokenizer.eos_token
pad_id = tokenizer.encode(pad)

train_input_ids_batch = []
train_attention_mask_batch = []
train_token_type_ids_batch = []

data = {
    "facts": [],
    "questions": ['What is the most frequent letter?', 's']
}

letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

num_facts = 8
max_length = (1024 - 7) // num_facts
for i in range(num_facts):
    sequence = random.choices(letters, k=max_length)
    data["facts"].append(" ".join(sequence))

for i, fact in enumerate(data["facts"]):
    in_txt = ""
    out_txt = fact
    
    ids1 = tokenizer(in_txt, return_tensors="pt")["input_ids"]
    ids2 = tokenizer(out_txt, return_tensors="pt")["input_ids"]

    n_mask = max_length - ids1.size(1) - ids2.size(1)
    assert n_mask >= 0, (max_length, ids1.size(1), ids2.size(1))
    padding = torch.LongTensor(pad_id * n_mask).unsqueeze(0)

    input_ids = torch.cat((ids1, ids2, padding), dim=1)
    attention_mask = torch.LongTensor(
        [1] * (ids1.size(1) + ids2.size(1)) + [0] * n_mask).unsqueeze(0)
    token_type_ids = torch.LongTensor(
        [0] * ids1.size(1) + [1] * ids2.size(1) + [0] * n_mask).unsqueeze(0)

    assert input_ids.size(1) == attention_mask.size(
        1) == token_type_ids.size(1) == max_length
    
    train_input_ids_batch.append(input_ids)
    train_attention_mask_batch.append(attention_mask)
    train_token_type_ids_batch.append(token_type_ids)

train_input_ids = torch.cat(train_input_ids_batch, dim=0)
train_attention_mask = torch.cat(train_attention_mask_batch, dim=0)
train_token_type_ids = torch.cat(train_token_type_ids_batch, dim=0)

features = {
    "input_ids": train_input_ids,
    "attention_mask": train_attention_mask,
    "token_type_ids": train_token_type_ids,
    "labels": train_input_ids
}

In [4]:
dev_input_ids_batch = []
dev_attention_mask_batch = []
dev_token_type_ids_batch = []

qa = data["questions"]
in_txt = qa[0]
print(in_txt)
out_txt = qa[1]
print(out_txt)

ids1 = tokenizer(in_txt, return_tensors="pt")["input_ids"]
ids2 = tokenizer(out_txt, return_tensors="pt")["input_ids"]

n_mask = max_length - ids1.size(1) - ids2.size(1)
assert n_mask >= 0, (max_length, ids1.size(1), ids2.size(1))
padding = torch.LongTensor(pad_id * n_mask).unsqueeze(0)

input_ids = torch.cat((ids1, ids2, padding), dim=1)
attention_mask = torch.LongTensor(
    [1] * (ids1.size(1) + ids2.size(1)) + [0] * n_mask).unsqueeze(0)
token_type_ids = torch.LongTensor(
    [0] * ids1.size(1) + [1] * ids2.size(1) + [0] * n_mask).unsqueeze(0)

assert input_ids.size(1) == attention_mask.size(
    1) == token_type_ids.size(1) == max_length

dev_input_ids_batch.append(input_ids)
dev_attention_mask_batch.append(attention_mask)
dev_token_type_ids_batch.append(token_type_ids)

dev_input_ids = torch.cat(dev_input_ids_batch, dim=0)
dev_attention_mask = torch.cat(dev_attention_mask_batch, dim=0)
dev_token_type_ids = torch.cat(dev_token_type_ids_batch, dim=0)

dev_features = {
    "input_ids": dev_input_ids,
    "attention_mask": dev_attention_mask,
    "token_type_ids": dev_token_type_ids,
    "labels": dev_input_ids
}

What is the most frequent letter?
s


In [5]:
model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda:5")
# model = torch.compile(model)

model_params = []

for idx, (name, param) in enumerate(model.named_parameters()):
    if param.requires_grad:
        model_params.append({"params": param, "lr": 1e-5})

inner_optimizer = torch.optim.Adam(model_params, lr=1e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=3e-5, momentum=0.9)
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
optimizer = inner_optimizer

In [6]:
import time

model.train()
context = " ".join(data["facts"])
context += f"\n {data['questions'][0]}"
tokenized = tokenizer([context], return_tensors="pt").to("cuda:5")
tokenized["input_ids"].shape

torch.Size([1, 1024])

In [8]:
from tqdm import tqdm

elapsed_times = []
for n in tqdm(range(100)):
    start = time.time()

    #for i in range(18):
    output = model(**tokenized, labels=tokenized["input_ids"], use_cache=True)

    end = time.time()

    elapsed = end - start
    elapsed_times.append(elapsed)

avg_elapsed = sum(elapsed_times) / len(elapsed_times)
print(f"Avg Runtim: {avg_elapsed}s")

100%|██████████| 100/100 [00:12<00:00,  7.85it/s]

Avg Runtim: 0.1264362144470215s


In [9]:
print(f"Avg Runtim: {avg_elapsed}s")

Avg Runtim: 0.1264362144470215s


In [11]:
import higher

for k in features:
    features[k] = features[k].to("cuda:5").long()

for k in dev_features:
    dev_features[k] = dev_features[k].to("cuda:5").long()

elapsed_times = []
for n in tqdm(range(100)):
    with higher.innerloop_ctx(
        model, optimizer,
        copy_initial_weights=False,
        track_higher_grads=False) as (fmodel, diffopt):

        start = time.time()

        for j in range(1):
            outputs = fmodel(**features, use_cache=True)
            inner_loss = outputs['loss']
            diffopt.step(inner_loss)
        
        with torch.no_grad():
            #for _ in range(18):
            outputs = fmodel(**dev_features, use_cache=True)

        end = time.time()

        elapsed = end - start
        elapsed_times.append(elapsed)

avg_elapsed = sum(elapsed_times) / len(elapsed_times)
print(f"Avg Runtim: {avg_elapsed}s")

100%|██████████| 100/100 [02:35<00:00,  1.56s/it]

Avg Runtim: 1.5005029726028443s


In [12]:
print(f"Avg Runtim: {avg_elapsed}s")

Avg Runtim: 1.5005029726028443s


In [ ]:
from tqdm import tqdm 
from meta_kg.optimizer import LSLRSchedular

num_iterations = 5
train_loss = []
train_loop = tqdm(range(num_iterations))

schedular = LSLRSchedular(
    inner_optimizer,
    num_inner_iter=num_iterations,
    init_lr=1e-5
)
schedular.initialization(model.named_parameters())


def compute_loss(features, outputs):
    logits = outputs["logits"][..., :-1, :].contiguous()
    labels = features["input_ids"][..., 1:].contiguous()
    label_mask = features["token_type_ids"][..., 1:].contiguous()

    loss_fct = nn.CrossEntropyLoss(reduction="none")
    losses = loss_fct(
        logits.view(-1, logits.size(-1)),
        labels.view(-1)
    )
    losses = losses.view(logits.size(0), logits.size(1)) * label_mask
    loss = torch.sum(losses, axis=1) / torch.sum(label_mask, axis=1)
    loss = torch.mean(loss)

model.train()
for i in train_loop:
    outputs = model(**features)
    loss = outputs['loss']
    train_loss.append(loss.item())
    train_loop.set_description(f"train_loss: {loss.item():.4f}")
    loss.backward()
    schedular.step(model.named_parameters(), i)
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
with torch.no_grad():
    model.eval()
    outputs = model(**features)
    logits = outputs["logits"][..., :-1, :].contiguous()
    label_mask = features["token_type_ids"][..., 1:].contiguous()
    norm_logits = torch.softmax(logits, dim=-1)
norm_logits.size()

In [ ]:
def get_token_prob(prob_locs, norm_logits, input_ids, k=10):
    topk_tokens_seq = []
    for loc in prob_locs:
        token = tokenizer.decode(input_ids[loc])
        topk_prob = torch.topk(norm_logits[loc], k=k)
        topk_tokens = []
        for idx in topk_prob.indices:
            gen_token = tokenizer.decode(idx.unsqueeze(0))
            prob = round(norm_logits[loc][idx].item() * 100, 3)
            topk_tokens.append((token.strip(), gen_token.strip(), prob))
        topk_tokens_seq.append(topk_tokens)
    return topk_tokens_seq

batch_topk_tokens = []
for i in range(norm_logits.size(0)):
    input_ids = features["input_ids"][i][1:]
    norm_logits_item = norm_logits[i]* label_mask[i].unsqueeze(-1)
    prob_locs = torch.nonzero(norm_logits_item)[:, 0].unique()
    topk_tokens = get_token_prob(prob_locs, norm_logits_item, input_ids)
    batch_topk_tokens.append(topk_tokens)


In [ ]:
import higher
from tqdm import tqdm 

model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda:2")
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
num_iterations = 5
loss_fct = nn.CrossEntropyLoss(reduction="none")

model.train()
train_loop = tqdm(range(num_iterations))
for i in range(num_iterations):
    with higher.innerloop_ctx(
        model, optimizer,
        copy_initial_weights=False,
        track_higher_grads=True) as (fmodel, diffopt):

        for j in range(1):
            outputs = fmodel(**features)
            inner_loss = outputs['loss']
            # logits = outputs["logits"][..., :-1, :].contiguous()
            # labels = features["input_ids"][..., 1:].contiguous()
            # label_mask = features["token_type_ids"][..., 1:].contiguous()

            # inner_losses = loss_fct(
            #     logits.view(-1, logits.size(-1)),
            #     labels.view(-1)
            # )
            # inner_losses = inner_losses.view(logits.size(0), logits.size(1)) * label_mask
            # inner_loss = torch.sum(inner_losses, axis=1) / torch.sum(label_mask, axis=1)
            # inner_loss = torch.mean(inner_loss)
            diffopt.step(inner_loss)
            print(f"inner_loss: {inner_loss.item():.4f}")

        outputs = fmodel(**dev_features)
        logits = outputs["logits"][..., :-1, :].contiguous()
        labels = dev_features["input_ids"][..., 1:].contiguous()
        label_mask = dev_features["token_type_ids"][..., 1:].contiguous()

        losses = loss_fct(
            logits.view(-1, logits.size(-1)),
            labels.view(-1)
        )
        losses = losses.view(logits.size(0), logits.size(1)) * label_mask
        loss = torch.sum(losses, axis=1) / torch.sum(label_mask, axis=1)
        loss = torch.mean(loss)
        print(f"loss: {loss.item():.4f}")

        # loss.backward()
        # optimizer.step()
        # optimizer.zero_grad()

